# MCT4001 Scientific Computing in Python Session 7
![mct-banner](https://raw.githubusercontent.com/wiki/MCT-master/Guides/assets/img/mct-banner.jpg)

In [ ]:
# enabling auto completion
%config IPCompleter.greedy=True

# importing packages
import os

## 1. Python and Pure Data Integration

In order to run the following code cells in on your computer you must update the path to Pure Data executable.

In the following cells we simply run Pure Data via [command line](https://puredata.info/docs/faq/commandline). Instead of writing the commands in a terminal, we invoke the terminal commands from python using the os package (in particular os.system()).

Before running the cells below, get familiar with the Pure Data patches used in this Jupyter notebook, stored in the folder ./files/puredata/

**Known issues**: the following cells should work well on OSX and Linux (you still have to update the Pure Data path for your computer in the next cell). However, on Windows, the *-send* flag may require minor adjustments (e.g. replace single quotes with double quotes or viceversa, etc.).

In [ ]:
# this must be updated to the actual path on your computer
pd_executable = '/Applications/Pd-0.52-2.app/Contents/Resources/bin/pd' #for OSX Pd 0.52-2
#pd_executable = 'C:\Program Files\pd\bin\pd.exe' #for Windows
#pd_executable = '/usr/local/bin/pd ' #for Linux

# running PD - you have to close it manually after running this cell
os.system(pd_executable)

In [ ]:
# running 5secSineToDac.pd
pd_patch = 'files/puredata/5secSineToDac.pd'
command = pd_executable + ' -open ' + pd_patch #for convenience, combining the command in a single string
print(command) #checking the correctness of the command string before using it in os.system()
os.system(command)

In [ ]:
# running 5secSineToDac.pd without GUI
pd_patch = 'files/puredata/5secSineToDac.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui'
print(command)
os.system(command)

In [ ]:
# running 5secSineToDac.pd without GUI passing frequency value
pd_patch = 'files/puredata/5secSineToDacVar.pd'
command = pd_executable + ' -open ' + pd_patch + ' -send "; freqV 1000"' + ' -nogui'
print(command)
os.system(command)

In [ ]:
# running 5secSineToWav.pd without GUI
pd_patch = 'files/puredata/5secSineToWav.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui'
print(command)
os.system(command)

In [ ]:
# running 5secSineToWav.pd without GUI offline as batch process
# (i.e. as fast as possible without being bounded by the audio sampling rate)
pd_patch = 'files/puredata/5secSineToWav.pd'
command = pd_executable + ' -open ' + pd_patch + ' -nogui' + ' -batch'
print(command)
os.system(command)

In [ ]:
# running SineToWavVar.pd without GUI offline as batch process passing values for frequency, duration and filename
pd_patch = 'files/puredata/SineToWavVar.pd'
send =  ' -send "; freqV 5000; nameV myfile.wav; durV 10000"'
command = pd_executable + ' -open ' + pd_patch + send + ' -nogui' + ' -batch'
print(command)
os.system(command)

In [ ]:
# creating a loop that generate a collection (curpus) of wave files with different frequencies
# files be further loaded and processed in Python either within the loop or (better) after the loop
pd_patch = 'files/puredata/SineToWavVar.pd'

for i in range(100,500,10):
    send =  ' -send "; freqV ' + str(i) + '; nameV out' + str(i) + '.wav; durV 10000"'
    command = pd_executable + ' -open ' + pd_patch + send + ' -nogui' + ' -batch'
    print(command)
    os.system(command)

#### Task 1

Take a Pure Data patch you have previously developed. Modify the patch so that it stops after 5 seconds, saves the output audio to a wave file (file name passed from Python), and it can take one argument (one value used for the sound generation, perhaps generated from a random number) from the command line.

Run the patch from Python in batch mode. When the Pure Data execution ends, open the generated file with librosa and display the waveform or spectrogram, as well as playback the audio using the IPython display audio widget.

## 2. Python OSC

Documentation available [here](https://python-osc.readthedocs.io/en/latest/).

#### Python OSC client example

In [ ]:
# run this cell after openning files/puredata/OSCserver.pd
import time
from pythonosc import udp_client

# creating OSC client that will send messages to 127.0.0.1 or localhost - i.e. same machinne - on port 8002
client = udp_client.SimpleUDPClient('127.0.0.1', 8002)

for x in range(5):
    client.send_message('/my_address', x) #sending the index of the loop with specified OSC address
    time.sleep(1) #doig nothing for 1 secod

#### Python OSC server example

In [ ]:
# run this cell and then open files/puredata/OSCclient.pd, click on connect and send messages
from pythonosc import dispatcher
from pythonosc import osc_server

# creating a function that will handle the  OSC received data, just printing inn this case
def handler(address, args):
    print('address is',address)
    print('values are')
    print(args)
    print('')
    

# attaching the message handling function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("*", handler) #with the wildcard * the handler function will handle any message

#starting initializing and  starting  the OSC server
server = osc_server.ThreadingOSCUDPServer(('127.0.0.1', 8001), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
# closing the OSC server
# if we do not do this, next time we try to create a server listenign on the same port we will get an error
server.server_close()

#### Python OSC server-client example

The following python cell receives data from Pure Data via OSC, process the ddata, and send it back to Pure data via OSC. On both sides, we are using a OSC client and server. 

In [ ]:
# run this cell and then open files/puredata/OSCclient-server.pd, click on connect, send messages, and observe messages sent back

from pythonosc import dispatcher
from pythonosc import udp_client
from pythonosc import osc_server
from IPython import display
import math


# creating OSC client that will send messages to 127.0.0.1 or localhost - i.e. same machinne - on port 8002
client = udp_client.SimpleUDPClient('127.0.0.1', 8002)


# creating a functions that will handle the OSC received data
def slider1_handler(address, args):
    #display.clear_output(wait=True) #this function simply clears the standard, then we print the most recent received OSC message
    #print('S1', args)
    out = abs(args-1)
    client.send_message('/invslider1', out)
    
def slider2_handler(address, args):
    #display.clear_output(wait=True) #this function simply clears the standard, then we print the most recent received OSC message
    #print('S1', args)
    out = math.log(1+100*args)/math.log(100)
    #print(out)
    client.send_message('/logslider2', out)

# attaching the message handling function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/slider1*", slider1_handler) #attaching handler to dispatcher
dispatcher.map("/slider2*", slider2_handler) #attaching handler to dispatcher

# starting initializing and  starting  the OSC server
server = osc_server.ThreadingOSCUDPServer(('127.0.0.1', 8001), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()
    

In [ ]:
# closing the OSC server
# if we do not do this, next time we try to create a server listenign on the same port we will get an error
server.server_close()

#### Task 2

Extend the above example to add an extra OSC parameter sent from Pure Data to Python and from Python to Pure Data. This requires to modify the Pure Data patch OSCclient-server.pd as well.

#### Task 3

Work in groups, try to develop from scratch a system which takes any sensor data from your smartphone (e.g. from [Multisense OSC](https://play.google.com/store/apps/details?id=edu.polytechnique.multisense.release&hl=en&gl=US) or [OSChook](https://m.apkpure.com/oschook/com.hollyhook.oscHook) or similar - OSChook must be installed manually, it does not exist on the Playstore), to control anything within a Pure Data patch (it can be as basic as controlling amplitude and pitch of a sinusoidal oscillator). However, data must be sent from the smartphone to Python, which performs some basic processing (can be as simple as an addition and/or multiplication), then Python sends the data to Pure Data, which generates the sound. Alternatively you can opt for a non-realtime application (i.e. generation of  a wave file from Pure Data).


Consider running Pure Data and Python on two (or more) separate machines (if connected to UiO VPN or eduroam OSC packets goes through). Ensure (and monitor) that each system can keep up with the rate at which messages are received (i.e. complete the required processing before the next message is received). The rate at which messages are received and passed through the chain depends on the rate at which OSC messages are produced in the smartphone application. You can easily do that using the [time](https://docs.python.org/3/library/time.html) package in Python as described [here](https://stackoverflow.com/questions/7370801/how-do-i-measure-elapsed-time-in-python). Try to change the OSC message rate in the smartphone application and see if the system can still cope with that (by looking at the overall application behavior as well as at the time measurements).


### Considerations

In this notebook we have explored how to run Pure Data patches from Python (at audio rate generating audio signals sent to the soundcard output, as well as in batch mode).

If you are interested in doing the opposite, i.e. run Python within Pure Data, you can try using this [package](https://grrrr.org/research/software/py/) and/or simply executing command line instructions from Pure Data. Since Pure Data has strict real-time computation constraints (it has to generate audio samples as a metronome to fill up the output buffer of thesoundcard in a timely manner), it is not a great idea to integrate into it Python computation which may break/compromize such constraints. Calling Python routines from Pure Data in a non-blocking and non-synchronous manner by using communication protocol such as OSC is probably the best solution.